In [1]:
import pandas as pd

In [20]:
import pandas as pd

df_payments = pd.DataFrame()
df_providers = pd.DataFrame()
df_ex_rates = pd.read_csv('ex_rates.csv')

for i in range(1, 3):
    payments_data = pd.read_csv(f'payments_{i}.csv')
    providers_data = pd.read_csv(f'providers_{i}.csv')
    
    df_payments = pd.concat([df_payments, payments_data], ignore_index=True)
    df_providers = pd.concat([df_providers, providers_data], ignore_index=True)


In [21]:
df_payments

,eventTimeRes,amount,cur,payment,cardToken
0,2024-11-25 20:59:59,90000.00,UZS,7d1476c9f1eedcd4a2083cc399ec1c1b,7ba0e52864c29fe2fcc1977be1ffc8d5
1,2024-11-25 20:59:58,2700.00,RUB,6eb49e22afdd28083ff82cdff8068880,774d551c9fff62fbac6c4ef424af0446
2,2024-11-25 20:59:58,2000.00,RUB,0e2705d1083b67f2c5ea12de3b7ab182,a3481a01b54d0c96dd44200389888bc6
3,2024-11-25 20:59:58,27.27,AZN,c30b3850e85454145bcb0dda3d209d88,61e3567ab56a79911fb7bcc0cc13db64
4,2024-11-25 20:59:58,8.00,AZN,4af690ba4eebb6d4782d9a3051dc58d2,373e0a66a63b8661f10afd098c68aeab
...,...,...,...,...,...
393027,2024-11-25 21:00:04,154.00,TRY,df37ef69adeffbf13245515035211a6a,dedbb3a4a41549a49d00dff275d2fa33
393028,2024-11-25 21:00:02,90000.00,UZS,22fa8e82050c2a448785dad1dc8e56ce,28873fe44be85e056f0f551cbe95e1a8
393029,2024-11-25 21:00:01,8.00,AZN,2916273333ab3ed18d68704248f73ccb,2cb1592a04ec78b2a5e8320d0d9b5de7
393030,2024-11-25 21:00:00,9.00,AZN,d740af44a63e64d3b220cf3dcb5d553c,dc704af94496050774cb55cd53e995f5
